In [253]:
import numpy as np
import pandas as pd

with open('input.txt') as f:
    data = f.read()
  
#data = 
'''class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9'''

'class: 0-1 or 4-19\nrow: 0-5 or 8-19\nseat: 0-13 or 16-19\n\nyour ticket:\n11,12,13\n\nnearby tickets:\n3,9,18\n15,1,5\n5,14,9'

In [262]:
rules = data.split("\n\n")[0]
your_ticket = data.split("\n\n")[1]
your_ticket = np.array(your_ticket.split('\n')[1].split(','), dtype=int)
tickets = data.split("\n\n")[2]

rules = {text.split(":")[0] : text.split(": ")[1] for text in rules.splitlines()}
tickets = tickets.split(":\n")[1]

tickets = [[int(t) for t in ticket.split(",")] for ticket in tickets.splitlines()]

rules_dict = {}

for key, value in rules.items():
    
    ranges = value.split(" or ")
    r1 = ranges[0].split("-")
    r2 = ranges[1].split("-")
    
    arr1 = np.arange(int(r1[0]), int(r1[1]) + 1)
    arr2 = np.arange(int(r2[0]), int(r2[1]) + 1)
    
    arr = np.concatenate((arr1, arr2))
    
    rule = dict.fromkeys(arr, 0)
    rules_dict = {**rules_dict, **rule}
    rules[key] = set(arr)
    
rules_set = set(rules_dict)

inv_sum = 0
valid_tickets = []

for ticket in tickets:
    
    invalid_fields = set(ticket) - rules_set
    inv_sum += sum(invalid_fields)
    
    if len(invalid_fields) == 0:
        valid_tickets.append(ticket)

print(f'First {inv_sum}')
ticket_matrix = np.array(valid_tickets)

check = pd.DataFrame(columns = rules.keys())
check["field"] = np.arange(len(rules))
check.set_index("field", inplace=True)        

for field, rule in rules.items():
    for index, row in enumerate(ticket_matrix.T):
        
        correct = len(set(row) - rule) == 0
        check.loc[index, field] = correct
        
correct_id = pd.DataFrame(index = rules.keys(), columns = ['Correct Field'])

for index, row in check.loc[check.sum(axis=1).sort_values().index].iterrows():
    fields_left = correct_id['Correct Field'].isna()
    correct_field = row.loc[(fields_left) & (row == True)]
    
    correct_id.loc[correct_field.index[0]] = index
    
deps = list(correct_id.loc[correct_id.index.str.contains('departure'), 'Correct Field'])
print(f'Second: {np.array(your_ticket)[deps].prod()}')

First 24110
Second: 6766503490793
